# [Tensorflow - Help Protect the Great Barrier Reef](https://www.kaggle.com/c/tensorflow-great-barrier-reef)
> Detect crown-of-thorns starfish in underwater image data

## Notebooks:
- Train: [object-detection-by-cys-train](https://www.kaggle.com/code/cuiyushuai/object-detection-by-cys-train)

- Infer: [object-detection-by-cys-infer](https://www.kaggle.com/code/cuiyushuai/object-detection-by-cys-infer)

<br><br>
# Install Libraries
> 安装库

- -q选项用于将安装过程设为静默模式
- -U选项用于确保安装最新版本的包
- "wandb"是Weights & Biases（W&B）的缩写，是一个用于跟踪和可视化机器学习实验的工具
- bbox-utility"是一个用于边界框（bounding box）操作和计算的实用工具包，常用于目标检测和图像分割任务中

In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility

<br><br>
# Import Libraries
> 导入库

- `numpy`: 用于数值计算的基础库

- `tqdm`: 提供可视化的进度跟踪（在循环中显示进度条）

- `pandas`: 用于数据处理和分析的强大工具

- `os`: 提供访问操作系统功能的接口

- `cv2`: 导入 `OpenCV` 库，它是一个用于CV任务的开源库，提供图像和视频处理功能

- `matplotlib`: 用于绘图和数据可视化的库

- `glob`: 提供了查找文件路径的功能，可以根据通配符匹配模式获取文件列表

- `shutil`: 提供了一些高级的文件和目录操作函数，比如复制、移动和删除等

- `sys.path.append`: 将路径添加到Python解释器的搜索路径中，为了导入指定路径下的库

- `joblib`: 导入函数，用于并行执行任务

- `IPython.display`: 用于在Jupyter Notebook中显示输出结果

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display

<br><br>
# Key-Points
> 关键点

* 与以往的目标检测比赛有所不同，参赛者必须使用**python时序API**提交预测
* 每行预测都需要包含图像的所有边界框。提交格式采用**COCO**格式，即 `[x_min、y_min、width、height]`
* 比赛的评价指标采用F2指标，它容忍一些假正例（FP），以确保较少海星被错过。这意味着解决假反例（FN）比假正例（FP）更重要
$$F2 = 5 \cdot \frac{precision \cdot recall}{4\cdot precision + recall}$$

<br><br>
# Introduction to WandB
<img src="https://camo.githubusercontent.com/dd842f7b0be57140e68b2ab9cb007992acd131c48284eaf6b1aca758bfea358b/68747470733a2f2f692e696d6775722e636f6d2f52557469567a482e706e67" width=600>

Weights & Biases（W&B）是一个MLOps（机器学习运营）平台，用于跟踪我们的实验。WandB提供了实验跟踪、数据集版本控制和模型管理的功能，帮助我们更快地构建更好的模型。

W&B的一些实用功能如下：

- 跟踪、比较和可视化机器学习实验

- 将实时指标、终端日志和系统统计信息通过集中式仪表板进行流式传输

- 解释模型工作原理、展示模型版本的改进图表、分析错误、展示达到阶段性的进展

## Import WandB
下面这段代码的目的是通过W&B库进行登录和身份验证。它首先尝试从"kaggle_secrets"模块中获取W&B访问令牌，如果失败则要求用户手动提供该访问令牌。

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

<br><br>
# Meta Data
> 元数据，对数据集中元素的介绍

* `train_images/` - 包含训练集照片的文件夹，照片的命名形式为 `video_{video_id}/{video_frame}.jpg`.
* `[train/test].csv` - 图像的元数据。与其他测试文件一样，在提交笔记本之前，大部分测试元数据只能在笔记本中使用，只有前几行可供下载
* `video_id` - 图像所属视频的ID编号。数据集中的图片来源于三个视频的抽帧，这个参数表明图片来自于哪个视频
* `video_frame` - 视频中图像的帧号。从潜水员浮出水面开始，帧号中偶尔会出现空缺
* `sequence` - 给出视频中无间隙子集的 ID。序列 ID 并非有序排列
* `sequence_frame` - 给出序列中的帧号。
* `image_id` - 图像的 ID 代码，格式为 `{video_id}-{video_frame}`
* `annotations` - 检测到的海星的的边界框，可直接用 Python 评估的字符串格式表示。与将、提交的预测使用的格式不同。在 test.csv 中不可用。边界框由图像左下角的像素坐标"(x_min, y_min) "及其像素 "宽 "和 "高 "来描述（COCO 格式）。
 
 
 在中文语境下，“annotations”直译为“注释”，所以后文中可能用“**注释**”、“**边界框**”或“**标注框**”来表示图片中**对海星的选择框**

In [ ]:
# 定义一个全局变量，后面会用于划分训练集和验证集
FOLD      = 1
# 图像的尺寸
DIM       = 3000 
# 所使用的目标检测的模型，yolov5s是YOLOv5的最小版本
MODEL     = 'yolov5s'
# 批次大小
BATCH     = 4
# 训练的轮数
EPOCHS    = 15
# 优化器使用Adam
OPTMIZER  = 'Adam'
# Weights & Biases（W&B）的项目名称，这里设置为'great-barrier-reef'，用于在W&B中进行记录和跟踪实验
PROJECT   = 'great-barrier-reef' # w&b in yolov5
# 训练任务的名称，用于在W&B中进行记录和跟踪实验。命名方式为模型名称、图像尺寸和折编号的组合。
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

# 一个布尔值，表示是否删除没有边界框的图像
REMOVE_NOBBOX = True # remove images with no bbox
# 根目录路径，指项目的根目录路径
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# 图像保存的目录路径
IMAGE_DIR = '/kaggle/images' # directory to save images
# 标签保存的目录路径
LABEL_DIR = '/kaggle/labels' # directory to save labels

## Create Directories
> 创建文件目录

根据上面定义的目录路径，使用mkdir命令创建文件夹

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

## Get Paths
> 获取路径

In [ ]:
# 训练集
# 读取train.csv
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
# 数据集的图片地址
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
# IMAGE_DIR是新建的文件夹，用于存放图片
df['image_path']  = f'{IMAGE_DIR}/'+df.image_id+'.jpg'
# LABEL_DIR是新建的文件夹，用于存放标签
df['label_path']  = f'{LABEL_DIR}/'+df.image_id+'.txt'
# annotations表示边界，应用eval从字符串格式转换为列表格式
df['annotations'] = df['annotations'].progress_apply(eval)
# 展示前两行数据，查看数据信息
display(df.head(2))

## Number of BBoxes
> 查看多少图片有标注框

通过前面对数据集的查看，我们发现前两个数据的 `annotations` 都为空，也就是没有标注框，下面我们查看一下有多少数据有标注

In [ ]:
# 使用progress_apply函数将lambda函数应用于df['annotations']列的每个元素
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
# 计算有多少图像没有边界框，有多少图像有边界框，并将数值转换为百分比
data = (df.num_bbox>0).value_counts(normalize=True)*100
# 输出
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

<br><br>
# Clean Data
> 清洗数据

在这个笔记本中，我们只选用**有边界框**(`~5k`)的数据进行训练，当然也可以使用**全部数据**(`~23k`)进行训练，但其中大部分图像没有任何标签。因此，只使用**bboxed-images**会更容易进行实验。

在下面这段代码中，由于 `REMOVE_NOBBOX` 属性先前已经定义为 `True` ，代码将筛选出 `bbox` 数量大于0的行，并将筛选结果存储在变量df中

In [ ]:
if REMOVE_NOBBOX:
    df = df.query("num_bbox>0")

<br><br>
# Write Images
> 写数据，将图片复制到新的目录

由于 `/kaggle/input` 目录没有YOLOv5所需的**写入权限**，我们要将图片复制到当前目录，即 `/kaggle/working` 。

我们可以使用使用**并行计算**组件**Joblib**来加快这一过程。

In [ ]:
# 将图片文件复制到新的目录下
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

In [ ]:
# 从old_image_path列中提取所有的图像路径，并将结果存储在image_paths变量中
image_paths = df.old_image_path.tolist()

# 调用Parallel函数，将make_copy函数应用于image_paths中的每个图像路径
# n_jobs=-1表示使用所有可用的CPU内核
# backend='threading'表示使用多线程而不是多进程
# tqdm函数用于显示进度条
# delayed函数用于将make_copy函数应用于image_paths中的每个图像路径
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

<br><br>
# Helper
> 辅助工具

* `coco2yolo`, `coco2voc`, `voc2yolo` - 用于在不同的标注格式之间进行转换
* `draw_bboxes` - 用于在图像上绘制边界框
* `load_image` - 用于加载图像
* `clip_bbox` - 用于裁剪边界框，使其适应图像的边界
* `str2annot` - 用于将字符串格式的边界框注释转换为字典格式
* `annot2str` - 用于将字典格式的边界框注释转换为字符串格式

In [ ]:
from bbox.utils import coco2yolo, coco2voc, voc2yolo
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

# 参数annots表示边界框的注释列表
def get_bbox(annots):
    # 注释字典的值转换为列表，并将结果存储在bboxes变量中
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

# 参数row表示数据集中的一行数据
def get_imgsize(row):
    # 函数用imagesize.get函数获取该图像的宽度和高度，宽度和高度被添加到row字典中
    row['width'], row['height'] = imagesize.get(row['image_path'])
    # 返回更新后的row字典
    return row

# 设定随机数生成器的种子为32，以确保每次运行代码时产生相同的随机数序列
np.random.seed(32)

# 这一行代码使用列表推导式生成一个包含一个随机颜色元组的列表
# 列表中只有一个元素，因为循环的范围是range(1)，这样就只生成了一个随机颜色
# 这些随机颜色可以用于在图像上绘制边界框时进行标记或区分不同的边界框
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

## Create BBox
> 在数据集中创建bbox属性，存储每个“注释”的位置、宽度、高度

In [ ]:
# 将DataFrame中的annotations列的每个元素应用于get_bbox函数
# progress_apply函数，在应用函数时显示一个进度条
# get_bbox函数将每个注释数据转换为边界框的列表，并将结果赋值给新的名为bboxes的列
# 最终，DataFrame中的每个行都将具有一个bboxes列，其中包含了对应的边界框列表
df['bboxes'] = df.annotations.progress_apply(get_bbox)
# 展示前两行，观察df数据中是否已经包含了bboxes属性
df.head(2)

## Get Image-Size
> 数据集中的所有图片具有相同的尺寸，[Width, Height] =  `[1280, 720]`

In [ ]:
# 为数据集添加
df['width']  = 1280
df['height'] = 720
# 展示前两行
display(df.head(2))

<br><br>
# Create Labels
> 创建标签

我们需要将标签导出为**YOLO**格式，每幅图像一个 `*.txt` 文件（如果图像中没有海星目标，则不需要 `*.txt` 文件）。 `*.txt` 文件的规格如下：

* 每个对象一行
* 每行的格式为： [x_center, y_center, width, height] 
* 方框坐标必须是**归一化**的 `xywh` 格式（ `0 ~ 1` ）。如果方框以像素为单位，则 `x_center` 和 `weight` 除以图像宽度， `y_center` 和 `height` 除以图像高度
* 标签号**从0开始**

> 我们已经得到的 `bbox` 的格式为 `COCO` ，即 `[x_min、y_min、width、height]` 。由于要在后续使用 `YOLOv5` ，我们需要将 `COCO` 格式转换为 `YOLO` 格式。

&nbsp;

下面这段代码的目的是将数据集中的**边界框数据转**换为 `YOLO` 格式的标签，并将每个图像的标签数据保存到对应的 `*.txt` 文件中。存入文件的

In [ ]:
# cnt用于记录没有标注框的图片数量
cnt = 0
# 用于存储所有的标注框
all_bboxes = []
# 用于存储标注框的相关信息
bboxes_info = []
# 遍历所有的图片，生成YOLO格式的标注文件。tqdm用于显示进度条
for row_idx in tqdm(range(df.shape[0])):
    # 获取对应行的数据
    row = df.iloc[row_idx]
    # 获取图片的相关信息
    # 高度
    image_height = row.height
    # 宽度
    image_width  = row.width
    # 图片的标注框
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    # 标注框的数量
    num_bbox     = len(bboxes_coco)
    # 创建一个长度为num_bbox的列表，其中每个元素都是字符串'cots'，用于表示边界框的类别名称
    names        = ['cots']*num_bbox
    # 创建一个形状为(num_bbox, 1)的NumPy数组，所有元素都是字符串'0'，用于表示边界框的标签
    labels       = np.array([0]*num_bbox)[..., None].astype(str)
    
    ## Create Annotation(YOLO)
    # 创建YOLO格式的标注框
    # 打开row.label_path，这是上文中定义过的文件夹，用于存储标注文件
    with open(row.label_path, 'w') as f:
        # 如果该图片没有标注框，则将该图片的标注文件置为空字符串，并将cnt加1
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        # 将COCO格式的边界框转换为VOC格式的边界框
        bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        # 将边界框的坐标裁剪到图像边界内
        bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
        # 将VOC格式的边界框转换为YOLO格式的边界框
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        # 将YOLO格式的边界框添加到all_bboxes列表中，并将边界框的数据类型转换为浮点型
        all_bboxes.extend(bboxes_yolo.astype(float))
        # 将图片的相关信息添加到bboxes_info列表中
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]]*len(bboxes_yolo))
        # 将标签和边界框数据按列方向连接，形成注释数据
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        # 将注释数据转换为字符串格式
        string = annot2str(annots)
        # 将字符串格式的注释数据写入到文件中
        f.write(string)
# 输出没有标注框的图片数量
print('Missing:',cnt)

<br><br>
# Create Folds
> 根据图片出自的三个视频，创建三个文件夹,起到分组的效果。这是因为每个视频中的样本数量不一样，这样会在**交叉验证**中产生很大的误差

在下面这段代码中，我们使用 `sklearn.model_selection` 模块中的 `GroupKFold` 类来进行**分组交叉验证**。

In [ ]:
# 导入GroupKFold类，该类用于分组交叉验证
from sklearn.model_selection import GroupKFold
# 创建一个GroupKFold对象kf，并将参数n_splits设置为3。这表示将数据分为3组进行交叉验证
kf = GroupKFold(n_splits = 3)
# 对DataFrame df 进行重置索引，将索引重新设置为默认的数字索引
df = df.reset_index(drop=True)
# 在DataFrame df 中创建一个名为fold的新列，并将其初始值设为-1。该列将用于存储每个样本所属的组号
df['fold'] = -1
# 使用kf.split方法对DataFrame进行分割，并在每次迭代中获取训练集索引train_idx和验证集索引val_idx
# 同时使用enumerate函数迭代得到折号fold和索引。
for fold, (train_idx, val_idx) in enumerate(kf.split(df, groups=df.video_id.tolist())):
    # 将验证集索引val_idx所对应的行的fold列的值设为当前组号fold。这样，每个样本都被分配到了对应的组号
    df.loc[val_idx, 'fold'] = fold
# 展示每一组中样本的数量
display(df.fold.value_counts())

<br><br>
# BBox Distribution
> BBox 分配

下面这段代码的功能是根据之前生成的 `bboxes_info` 和 `all_bboxes` 数据，创建一个新的DataFrame `bbox_df` ，并进行数据类型转换、计算面积以及与原始数据的合并操作，以便后续在目标检测模型训练中使用。

In [ ]:
# 使用np.concatenate函数将bboxes_info和all_bboxes按列方向连接起来，创建一个新的NumPy数组
# bbox_df DataFrame 包含图像ID、视频ID、视频中无间隙子集的ID、边界框的中心坐标、宽度、高度
bbox_df = pd.DataFrame(np.concatenate([bboxes_info, all_bboxes], axis=1),
             columns=['image_id','video_id','sequence',
                     'xmid','ymid','w','h'])
# 将bbox_df中的xmid、ymid、w和h列转换为浮点数
bbox_df[['xmid','ymid','w','h']] = bbox_df[['xmid','ymid','w','h']].astype(float)
# 计算边界框的面积，为bbox_df添加area（面积）属性
bbox_df['area'] = bbox_df.w * bbox_df.h * 1280 * 720
# 将bbox_df与原始数据df中的image_id和fold列进行左连接。将组号(fold)信息添加到bbox_df中
bbox_df = bbox_df.merge(df[['image_id','fold']], on='image_id', how='left')
# 展示前两行数据
bbox_df.head(2)

## x_center & y_center
> 可视化“标注框”坐标[x, y]的分布情况

下面这段代码通过高斯核密度估计来**可视化边界框在二维平面上的密度分布情况**。它可以帮助我们理解边界框的聚集程度、分布形态以及具体的密度情况。

In [ ]:
# 导入gaussian_kde函数，该函数用于进行高斯核密度估计
from scipy.stats import gaussian_kde

# 将all_bboxes转换为NumPy数组
all_bboxes = np.array(all_bboxes)

# 从all_bboxes数组中获取第一列数据，即边界框的x坐标
x_val = all_bboxes[...,0]
# 从all_bboxes数组中获取第二列数据，即边界框的y坐标
y_val = all_bboxes[...,1]

# 计算点密度
# 使用np.vstack函数将x_val和y_val垂直堆叠，形成一个2行的数组
xy = np.vstack([x_val,y_val])
# 使用gaussian_kde函数对堆叠后的数组xy进行高斯核密度估计，得到密度结果z
# 这里将xy作为输入，并使用相同的输入数据进行估计
z = gaussian_kde(xy)(xy)

# 创建一个大小为(10, 10)的图形窗口，并返回图形对象fig和坐标轴对象ax
fig, ax = plt.subplots(figsize = (10, 10))
# ax.axis('off')
# 使用scatter函数绘制散点图
# c=z表示使用密度结果z作为颜色映射
# s=100表示散点的大小为100
# cmap='viridis'表示使用'viridis'颜色映射方案
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
# ax.set_xlabel('x_mid')
# ax.set_ylabel('y_mid')
# 显示绘制的图形
plt.show()

## width & height
> 可视化“标注框”宽度和高度的分布情况

通过下面这段代码，我们可以直观地了解边界框宽度和高度的分布情况。密度较高的区域颜色较深，而密度较低的区域将颜色较浅。这有助于我们发现宽度和高度的分布特征，例如聚集的区域、稀疏的区域以及整体的分布模式。

In [ ]:
# 从all_bboxes数组中获取第三列数据，即边界框的宽度
x_val = all_bboxes[...,2]
# 从all_bboxes数组中获取第四列数据，即边界框的高度
y_val = all_bboxes[...,3]

# 计算点密度，和上文中相同
xy = np.vstack([x_val,y_val])
z = gaussian_kde(xy)(xy)

# 绘图，和上文中相同
fig, ax = plt.subplots(figsize = (10, 10))
# ax.axis('off')
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
# ax.set_xlabel('bbox_width')
# ax.set_ylabel('bbox_height')
plt.show()

## Area
> 对“标注框”的面积分布情况进行可视化

* seaborn库提供了更高级的图表风格和绘图函数

* 不同的**fold（组号）**属性的值对应不同的颜色

In [ ]:
import matplotlib as mpl
# seaborn库是一个基于matplotlib的数据可视化库，提供了更高级的图表风格和绘图函数
import seaborn as sns

# 创建一个大小为(12, 6)的图形窗口，并返回图形对象f和坐标轴对象ax
f, ax = plt.subplots(figsize=(12, 6))
# 去除图表的上、右边框线
sns.despine(f)

# 使用seaborn库的histplot函数绘制直方图
sns.histplot(
    # bbox_df是直方图的数据源
    bbox_df,
    # 标注框的面积作为x轴，hue表示根据fold列的值对直方图进行分组，并用不同的颜色表示
    x="area", hue="fold",
    # 将分组的直方图以堆叠的形式展示
    multiple="stack",
    # 使用"viridis"颜色映射方案
    palette="viridis",
    # 表示直方图的边缘颜色为浅灰色
    edgecolor=".3",
    # 表示直方图的边缘线宽为0.5
    linewidth=.5,
    # 表示使用对数刻度
    log_scale=True,
)
# 设置x轴刻度的格式为标量格式
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
# 设置x轴的刻度位置为500、1000、2000、5000和10000
ax.set_xticks([500, 1000, 2000, 5000, 10000]);

<br><br>
# Visualization
> 将部分训练数据可视化

在下面这段代码中，我们找出部分**带有标注框**（这里的“带有”指数据集中的该图片有对应的标注数据）的图片，调用 `draw_bboxes` 函数绘制标注框，并将图片进行可视化。

In [ ]:

# 从数据集df中选择num_bbox大于0的样本，并随机抽样100个样本，将结果保存在df2中
# num_bbox表示被选择的样本要有标注框
df2 = df[(df.num_bbox>0)].sample(100)
# 定义变量y和x，分别表示子图的行数和列数
y = 3; x = 2
# 创建一个图形窗口，并设置其大小为12.8*x的宽度和7.2*y的高度
plt.figure(figsize=(12.8*x, 7.2*y))
# 遍历子图的索引，共有x*y个子图
for idx in range(x*y):
    # 获取df2中索引为idx的样本
    row = df2.iloc[idx]
    # 加载图像，图像路径存储在row.image_path中
    img           = load_image(row.image_path)
    # 获取图像的高度，存储在row.height中
    image_height  = row.height
    # 获取图像的宽度，存储在row.width中
    image_width   = row.width
    # 打开标签文件，标签文件路径存储在row.label_path中
    with open(row.label_path) as f:
        # 取标签文件内容，并将其转换为标注数组
        annot = str2annot(f.read())
    # 从标注数组中获取边界框的坐标信息
    bboxes_yolo = annot[...,1:]
    # 从标注数组中获取边界框对应的类别标签，并将其转换为整数列表
    labels      = annot[..., 0].astype(int).tolist()
    # 创建长度为边界框数量的类别名称列表，这里使用了"cots"作为类别名称
    names         = ['cots']*len(bboxes_yolo)
    # 在图形窗口中创建一个子图，行数为y，列数为x，当前子图的索引为idx+1
    plt.subplot(y, x, idx+1)
    # 使用draw_bboxes函数绘制带有边界框的图像，并使用imshow函数显示图像
    plt.imshow(draw_bboxes(img = img,
                           bboxes = bboxes_yolo, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = 'yolo',
                           line_thickness = 2))
    # plt.axis('OFF')：关闭坐标轴显示
    plt.axis('OFF')
# 自动调整子图布局，以确保子图之间的间距合适
plt.tight_layout()
# 显示绘制的图像
plt.show()

<br><br>
# Dataset
> 处理数据集

下面这段代码，将训练数据中fold为2和3的划分为训练集，fold为1的划分为验证集合

In [ ]:
# 创建空列表 train_files，用于存储训练集的样本文件路径
train_files = []
# 创建空列表 val_files，用于存储验证集的样本文件路径
val_files   = []
# FOLD事先定义为1，下面这两行，表示组号为2和3的作为训练集，组号为1的作为验证集
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
# 将训练集子数据集中的样本文件路径添加到 train_files 列表中。使用 unique 方法确保每个文件路径只被添加一次
train_files += list(train_df.image_path.unique())
# 将验证集子数据集中的样本文件路径添加到 val_files 列表中。使用 unique 方法确保每个文件路径只被添加一次
val_files += list(valid_df.image_path.unique())
# 返回 train_files 和 val_files 列表的长度，即训练集和验证集中样本的数量
len(train_files), len(val_files)

<br><br>
# Configuration
> 为YOLOv5的执行设立一系列的配置文件

## gbr.yaml
> 训练数据配置文件

该配置文件中包含的内容：
* 工作路径 `/kaggle/working` 
* 将训练集和验证集的图片路径存入对应的 `train.txt` 和 `val.txt` 两个 `txt` 文件
*  `nc` 表示目标检测任务中待检测的物体类别数量，我们的检测目标只有海星，在前文中命名为 `cots` ，所以 `nc = 1` 
*  `name` 指定了目标检测任务中的类别名，只有一个类别，名称为 `cots` 

In [ ]:
# 导入yaml模块，用于处理YAML文件
import yaml
# 设置工作路径
cwd = '/kaggle/working/'
# 打开“train.txt”文件，遍历train_files列表中的每个文件路径，并将其写入到“train.txt”文件中
with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path+'\n')
# 打开“val.txt”文件，遍历val_files列表中的每个文件路径，并将其写入到“val.txt”文件中
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in valid_df.image_path.tolist():
        f.write(path+'\n')
# 字典data是用于配制YOLOv5的配置文件
data = dict(
    # 工作路径
    path  = '/kaggle/working',
    # 训练集的txt文件
    train =  os.path.join( cwd , 'train.txt') ,
    # 验证集的txt文件
    val   =  os.path.join( cwd , 'val.txt' ),
    # nc表示目标检测任务中待检测的物体类别数量
    nc    = 1,
    # name指定了目标检测任务中的类别名，只有一个类别，名称为cots
    names = ['cots'],
    )

# 将数据字典写入YAML配置文件
# 打开gbr.yaml，在此文件中存储YAML格式的配置数据
with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    # yaml.dump函数将数据字典的内容以YAML格式写入文件中
    yaml.dump(data, outfile, default_flow_style=False)
# 读取并打印YAML配置文件内容
f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())

## hyp.yaml
> 超参数配置文件

该配置文件中包含的内容：
* 超参数
* 数据增强操作的参数

下面这段代码定义了一系列超参数和数据增强操作的参数，用于在训练模型时调整学习率和对图像进行增强处理，并定制模型训练过程中的行为。

In [ ]:
%%writefile /kaggle/working/hyp.yaml
# 初始学习率 (SGD=1E-2，Adam=1E-3)
lr0: 0.01
# 最终学习率为初始学习率的一部分 (lr0 * lrf)
lrf: 0.1
# # SGD 动量/Adam beta1
momentum: 0.937  
# 优化器的权重衰减，相当于5e-4
weight_decay: 0.0005
# 热身阶段的轮数（允许小数）
warmup_epochs: 3.0
# 热身阶段的初始动量
warmup_momentum: 0.8
# 热身阶段的初始偏置学习率
warmup_bias_lr: 0.1
# 框损失的权重
box: 0.05
# 类别损失的权重
cls: 0.5
# 类别损失 BCELoss 的正权重
cls_pw: 1.0
# 目标损失的权重（与像素比例相关）
obj: 1.0
# 目标损失 BCELoss 的正权重
obj_pw: 1.0
# IoU 训练阈值
iou_t: 0.20
# 锚框倍数阈值
anchor_t: 4.0
# 每个输出层的锚框数量
# anchors: 3
# 焦点损失的 gamma（efficientDet 默认 gamma=1.5）
fl_gamma: 0.0
# 图像 HSV 色调增强参数（分数）
hsv_h: 0.015
# 图像 HSV 饱和度增强参数（分数）
hsv_s: 0.7
 # 图像 HSV 亮度增强参数（分数）
hsv_v: 0.4
# 图像旋转角度（正负度数）
degrees: 0.0
 # 图像平移参数（正负分数）
translate: 0.10
# 图像缩放参数（正负增益）
scale: 0.5
# 图像剪切参数（正负度数）
shear: 0.0
# 图像透视变换参数（正负分数），范围在0-0.001之间
perspective: 0.0
 # 图像上下翻转概率
flipud: 0.5
# 图像左右翻转概率
fliplr: 0.5
# 图像马赛克概率
mosaic: 0.5
# 图像混合概率
mixup: 0.5
# 片段复制粘贴概率
copy_paste: 0.0

<br><br>
# YOLOv5 with WandB Integration
> YOLOv5和WandB的结合使用

<div align="center">

  <a href="https://ultralytics.com/yolov5" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/master/yolov5/v70/splash.png"></a>

</div>

* YOLOv5（You Only Look Once version 5）是一个目标检测算法，用于在图像中识别和定位物体
* WANDB（Weights & Biases）是一个用于实验跟踪、记录和可视化的工具
* YOLOv5负责训练和处理目标检测任务，而WANDB负责实验的记录、可视化和跟踪。在实验执行的过程中，可以在WANDB的仪表板中查看模型的训练指标、超参数、损失曲线、预测结果等信息，以便更好地理解模型的表现和效果。

In [ ]:
# 克隆YOLOv5仓库，这里用的是修改后的仓库，通过Wandb在训练过程中实时监控训练指标
!git clone https://github.com/cys02/yolov5-wandb.git yolov5
# 切换到yolov5目录下
%cd yolov5
# 安装依赖包，-qr表示安静模式，不输出安装的详细信息
%pip install -qr requirements.txt

import torch
import utils
# 检查GPU是否可用
display = utils.notebook_init()

<br><br>
# Training
> 调用YOLOv5中的 `train.py` 文件进行训练

其中部分参数在前文中已有定义，说明如下：
*  `DIM` - 表示图像的尺寸为3000
*  `BATCH` - 表示每个批次中有4个样本
*  `EPOCHS` - 表示训练轮数为15
*  `OPTMIZER` - 表示使用 `Adam` 优化器
*  `MODEL` - 表示使用 `yolov5s.pt` 作为初始权重文件
*  `PROJECT` - 先前定义为 `great-barrier-reef` 
*  `NAME` - 先前定义为 `f'{MODEL}-dim{DIM}-fold{FOLD}'` 

In [ ]:
!python train.py --img {DIM}\
--batch {BATCH}\
--epochs {EPOCHS}\
--optimizer {OPTMIZER}\
--data /kaggle/working/gbr.yaml\
--hyp /kaggle/working/hyp.yaml\
--weights {MODEL}.pt\
--project {PROJECT} --name {NAME} --entity cys02\
--exist-ok

<br><br>
# Overview
> WandB中展示的学习进程

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;"><a href="https://wandb.ai/cys02/great-barrier-reef">View the Complete Dashboard Here</a></span>

## train

![image-20230809145710611](https://raw.githubusercontent.com/cys02/ImageRepository/master/image-20230809145710611.png)

## metrics

![image-20230809145906616](https://raw.githubusercontent.com/cys02/ImageRepository/master/image-20230809145906616.png)

## val

![](https://raw.githubusercontent.com/cys02/ImageRepository/master/image-20230809145938142.png)

## System

![](https://raw.githubusercontent.com/cys02/ImageRepository/master/image-20230809150117336.png)



## Output Files
> 输出文件

使用 `ls` 命令查看输出路径下的文件

In [ ]:
# 指定输出路径
OUTPUT_DIR = '{}/{}'.format(PROJECT, NAME)
# 使用ls命令查看输出路径下的文件，例如训练模型的权重文件、日志文件等
!ls {OUTPUT_DIR}

<br><br>
# Class Distribution
> 查看类别分布情况

## labels_correlogram.jpg
汇总训练集数据的标签labels，并画出训练集数据标签 x, y, width, height 4个变量之间的关系图（线性或非线性，有无较为明显的相关关系）

In [ ]:
# 展示YOLOv5运行生成的labels_correlogram.jpg
plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/labels_correlogram.jpg'));

## labels.jpg
* 左上角：统计训练集数据每个类别数量直方图
* 右上角：把所有框的x和y中心值设置在相同位置看每个训练集数据每个标签框的长宽情况
* 左下角绘制 x, y 变量直方图来显示数据集的分布
* 右下角：绘制 width, height 变量直方图来显示数据集的分布

In [ ]:
# 展示YOLOv5运行生成的labels.jpg
plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/labels.jpg'));

<br><br>
# Batch Image
> 展示YOLOv5不同训练批次的样本图像，YOLOv5共输出了三个批次的样本图像

In [ ]:
# 展示三个批次的样本图像
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch0.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch1.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch2.jpg'))

## Ground Truth vs Predictions
> 实际标注信息和预测标注信息的对比

In [ ]:
# 创建一个3行2列的图像子图网格
# fig是整个图像画布对象，ax是包含3x2个子图对象的二维数组
# figsize参数指定了整个画布的尺寸，constrained_layout=True用于自动调整子图的位置，以防止它们重叠
fig, ax = plt.subplots(3, 2, figsize = (2*9,3*5), constrained_layout = True)
# 这个循环遍历每一行，对每一行的两个子图位置进行设置
# 在左边的子图位置，使用imshow()函数加载并显示预测图像
# 在右边的子图位置，使用imshow()函数加载并显示标签图像
# 用set_xticks([])和set_yticks([])来隐藏坐标轴刻度，使用set_title()来设置子图的标题，标题显示图像的文件名。
for row in range(3):
    ax[row][0].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg', fontsize = 12)
# 展示图像
plt.show()

<br><br>
# Result
> 查看训练结果

## Score vs Epoch
> 分数 vs 训练轮数，查看随着训练轮数的增加分数的变化情况

YOLOv5运行生成的 `result.png` 文件提供了损失函数（Loss）的曲线图、准确率、召回率等信息，帮助我们判断模型是否在训练中有收敛、过拟合、欠拟合等情况

In [ ]:
# 展示YOLOv5运行生成的results.png
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/results.png'));

## Confusion Matrix
> 混淆矩阵

混淆矩阵是一个二维矩阵，行表示真实标签，列表示预测标签。每个单元格中的值表示实际属于某个类别的样本被预测为另一个类别的数量。混淆矩阵展示的信息如下：

* True Positives (TP): 正确预测为正类别的数量。在目标检测中，表示正确检测到的目标数量。
* False Positives (FP): 错误预测为正类别的数量。在目标检测中，表示预测为目标但实际上并没有目标的数量。
* True Negatives (TN): 正确预测为负类别的数量。在目标检测中，表示正确判定为非目标的背景区域数量。
* False Negatives (FN): 错误预测为负类别的数量。在目标检测中，表示实际有目标但未能正确检测到的数量。

混淆矩阵的可视化以图像的形式展示这些数量。每个单元格的颜色或大小可以表示对应的数量。通过观察混淆矩阵，可以了解模型在不同类别上的表现情况，进而进行性能评估和调整。

在目标检测任务中，混淆矩阵可以帮助我们识别出模型在哪些类别上表现良好，哪些类别上可能存在问题，从而有针对性地进行改进和调优。

In [ ]:
# 展示YOLOv5运行生成的confusion_matrix.png
plt.figure(figsize=(12,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/confusion_matrix.png'));

## Metrics
> 模型性能评估指标

下面展示以下四个指标的曲线图：
* **F1 Score Curve ('F1')**: F1 分数是精确率（Precision）和召回率（Recall）的调和平均值，用于综合考虑模型的准确性和全面性。F1 分数在不同阈值下的变化曲线可以帮助我们找到一个平衡点，使得精确率和召回率之间达到最佳的平衡。

* **Precision-Recall Curve ('PR')**: 精确率-召回率曲线展示了在不同阈值下精确率和召回率的变化情况。精确率是模型预测为正类别中实际为正类别的比例，召回率是实际为正类别中被模型正确预测的比例。曲线上的每个点对应一个阈值，可以帮助我们根据任务需求选择合适的阈值。

* **Precision Curve ('P')**: 精确率曲线展示了在不同阈值下的模型精确率的变化情况。随着阈值的变化，模型的精确率可能会有所波动。我们可以根据任务的需求，选择适当的阈值来平衡精确率和召回率。

* **Recall Curve ('R')**: 召回率曲线展示了在不同阈值下的模型召回率的变化情况。召回率是指实际为正类别的样本中被模型正确预测的比例。曲线的变化可以帮助我们了解模型在不同召回率下的性能。

In [ ]:
# 展示YOLOv5运行生成的四张曲线图
for metric in ['F1', 'PR', 'P', 'R']:
    print(f'Metric: {metric}')
    plt.figure(figsize=(12,10))
    plt.axis('off')
    plt.imshow(plt.imread(f'{OUTPUT_DIR}/{metric}_curve.png'));
    plt.show()

<br><br>
# Remove Files
> 删除后续中不再需要的文件夹

In [ ]:
# 删除文件夹
!rm -r {IMAGE_DIR}
!rm -r {LABEL_DIR}